In [1764]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "compar.": "comparative",
    "compar": "comparative",
    "impér.": "imperative",
    "impér": "imperative",
    "diminutif": "diminutive",
    "vocatif": "vocative",
    "gén. fém.": "genitive_feminine",
    "superl.": "superlative"
}

In [1852]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [1765]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [2011]:
REST = {
    252: "ə-brɑ̃:n (Abrán) « avril » ; ɛ-bʹrʹᴜ: (oib­riughadh) « tra­vailler » ; ɔ-χlɑ̃:n (ochlán) « abatte­ment »",
    241: "lʲo:rgo:hɩnʹtʹ (leor­dhóthain) « quantité largement suffi­sante »"
}

Un groupe consonantique séparant une voyelle atone d’une voyelle tonique tend à appar­tenir à la syllabe suivante (à être explosif). C’est le cas dans les exemples signalés § 249, où les groupes sont traités, au point de vue du développe­ment de la voyelle svara­bhaktique, comme s’ils étaient initiaux, ce qui s’explique par le fait qu’on a là des groupes explosifs, comme à l’initiale ; ə-brɑ̃:n (Abrán) « avril » ; ɛ-bʹrʹᴜ: (oib­riughadh) « tra­vailler » ; ɔ-χlɑ̃:n (ochlán) « abatte­ment » ; etc. (ce qui n’empêche pas que la consonne ne soit unie à la voyelle précé­dente par jonction étroite « fester Anschluss »).

§ 253. Dans les composés, le sentiment de la composition peut maintenir la sylla­bation telle qu’elle était dans les termes isolés :

nʹα-vʹrʹɑ: ou nʹα-mʹrʹɑ̃:, non nʹαvʹ-rʹɑ: (neamhbhreágh) « laid » ; c’est ainsi qu’on a dans des composés comme ɑnə-hrᴇ̈:ᵊχtə (ana-thraochta) « très fatigué », des groupes qui sont communs à l’initiale, c’est-à-dire là où ils sont explosifs, mais ne se rencon­trent pas normale­ment en position médiane, où ils devraient être implosivo-explosifs.

§ 254. Dans les groupes de trois consonnes les deux premières appar­tiennent normale­ment à la première syllabe, la troisième à la seconde : ɑmp-ᵊlə (ampladh) « dévorer » ; αχt-ᵊrə (eachtra) « aventure » ; kʹinʹʃ-tʹər (cinstear) « fragments ». Il faut faire exception pour les groupes signalés § 232, 2º où h commence la deuxième syllabe : to:r-hnʲəχ, etc.

Par ailleurs, les deux dernières consonnes peuvent appartenir à la dernière syllabe, là où, si le groupe ne com­portait que deux consonnes, ces deux consonnes appartien­draient à la deuxième syllabe : gʹαl-χrɪ:ᵊχ (geal­chroidh­each) « au cœur droit, généreux » ; kɔs-tʹrʹλma·χ (costriom­ach) « pré­caution­neux » (litt. « aux pieds secs »).

§ 255. Début et fin de syllabe :

Une syllabe peut commencer soit par une voyelle, c’est-à-dire par une ouverture complète, sans resserre­ment glottal, l’attaque vocalique étant douce, mais cela seulement à l’initiale ; soit par une consonne ou par un groupe de consonnes. Un relevé portant sur 500 syllabes d’un texte de folklore (compte non tenu des chutes et élisions) m’a donné une moyenne de 72 syllabes pour 100 com­mençant par une consonne. Le groupe de consonnes qui commence la voyelle est, dans la plupart des cas, d’ouverture et de sonorité crois­santes, mais ce principe comporte deux excep­tions :

1º s, ʃ peuvent précéder des occlusives sourdes on assourdies : stɑd (stad) « arrête » (cf. § 215, 5º).

2º h peut précéder une liquide ou une nasale ; à l’initiale absolue h est sourde, mais la consonne qui suit est au moins partielle­ment sonore, si bien que le groupe, quoique d’ouverture décrois­sante, est de sonorité crois­sante : hn̬ɑ̃:ⁱvʹ ʃe· (shnáimh sé) « il nagea » ; dans le corps de la phrase, après sonore, h est en revanche sonore.

Le caractère parti­culier de la « sonorité », de l’h explique au reste cette anomalie, la sonorité de l’h « chuchée » (cf. § 87) étant moindre que celle de la nasale ou de la liquide, même partielle­ment assimilée.

La syllabe peut se terminer par une ouverture complète, par une consonne ou par un groupe de consonnes d’ouverture et de sonorité décrois­santes (cf. § 217 sq.) ; les règles qui viennent d’être résumées concer­nant la frontière sylla­bique expli­quent que la propor­tion des syllabes fermées soit consi­dérable : un relevé portant sur 500 syllabes d’un texte de folklore m’a donné 44 syllabes pour 100 finissant par une consonne. Ces chiffres ne tiennent pas compte des cas où une voyelle svara­bhaktique encore réduite a amorcé le développe­ment d’une nouvelle syllabe. Il va de soi que dans ce cas la disso­ciation du groupe a pour effet d’ouvrir la syllabe précé­dente : la tendance générale du parler est donc de diminuer le nombre des syllabes fermées.

§ 256. Malgré cela le trait le plus frappant qu’offre la consti­tution de la syllabe est l’absence d’un type normal ; tandis que (d’après M. de Groot, B. S. L., XXVII, p. 24) le latin de Caton offre une pro­portion de 91 syllabes pour 100 commen­çant par une consonne, de 62 pour 100 finissant par une voyelle, tandis qu’en français l’on a 95 syllabes pour 100 commen­çant par une consonne, 88 pour 100 finissant par une voyelle, dans notre parler la pro­portion est, approxi­mative­ment (dans la mesure où des relevés très res­treints donnent une indi­cation exacte), 72 syllabes pour 100 com­mençant par une consonne, et seulement 56 pour 100 finissant par une voyelle. Des diffé­rents types de syllabes possibles repré­sentés par ɑ:rd (árd) « haut » ; grɑ: (grádh) « amour » ; kʹαrt (ceart) « droit » ; e: (é) « lui », il n’en est donc aucun qui prédomine consi­dérable­ment.

§ 257. Accent syllabique :

La partie vocalique de la syllabe peut être constituée par une voyelle longue ou brève, de timbre uni ou compor­tant un glide, ou par une diph­tongue.

Lorsqu’on a une voyelle, il ne se pose pas de question au point de vue de l’accent : le maximum d’énergie articu­latoire concorde obli­gatoire­ment avec le maximum d’ouverture.

Il n’en va pas de même lorsqu’on a une diphtongue : nous avons vu qu’il existe deux types de diph­tongues ; d’ouverture décrois­sante : ᴀᴜ, ɔᴜ, aᴜ, ᴀɪ, ai, etc. et d’ouverture crois­sante (« fausses diph­tongues ») : i·ə, i·ɛ, ᴜ·ə, ᴜ·ɛ, ì:a. On sait que c’est sur la première partie de la diph­tongue que porte l’accent, à part quelques cas où se manifeste une tendance à faire passer l’accent sur la deuxième partie (cf. §§ 204 et 211).

Il en résulte que, dans les syllabes compor­tant une diph­tongue d’ouverture crois­sante, l’accent ne coïncide pas avec l’ouverture maximum, et est par ailleurs situé aussi près que possible du début de la syllabe.

§ 258. Durée de la syllabe :

Au point de vue de la durée, comme au point de vue de la consti­tution, la syllabe varie consi­dérable­ment en raison de la fréquence des groupes consonan­tiques, des oppo­sitions quanti­tatives qu’offre le système vocalique et du fait qu’une voyelle longue peut être suivie de groupes consonan­tiques aussi bien qu’une voyelle brève. Des exemples comme ceux-ci, où la division des syllabes coïncide avec celle des mots tɑ:ⁱmʹ ɛ sg̬rʹi:ᵊv' (táim ag scríobh­adh) « j’écris » ; ə sb̬lᴀᴜŋk (an splannc) « l’éclair » donnent une idée de la variété de forme et de l’inégalité de durée que présen­tent les unités sylla­biques du parler.



In [2012]:
# DATA[section] = []
section = 253
if not section in DATA:
    DATA[section] = []
text = REST[section].replace(" : ", " ; ")
text = text.replace("\u00ad", "")
text = text.replace("»", " »")
text = text.replace("  »", " »")
text = text.replace(", mais", " ; ")
extract(text, ";", 1)

22


In [1995]:
def split_trans(pos):
    if " ou " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")
    elif " et " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" et ")
    elif " à côté de " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" à côté de ")

for s in range(len(DATA[section])):
    if "transcription" in DATA[section][s]:
        split_trans(s)

In [2014]:
import json

if str(section) in DATA and section in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [2013]:
def french_to_note(pos):
    DATA[section][pos]["french_note"] = DATA[section][pos]["french"]
    del(DATA[section][pos]["french"])

def contrast(pos, pos1):
    if 'irish' in DATA[section][pos1]:
        DATA[section][pos]['contrast'] = DATA[section][pos1]['irish']
    else:
        DATA[section][pos]['contrast'] = DATA[section][pos1]['transcription']
    DATA[section][pos]['contrast_id'] = DATA[section][pos1]['id']

def contrast_next(pos):
    contrast(pos, pos+1)

def syllabified(item):
    if "-" in item["transcription"]:
        item["syllabified"] = item["transcription"]
        item["transcription"] = item["transcription"].replace("-", "")

def syllabify():
    for s in DATA[section]:
        syllabified(s)

syllabify()
# contrast(8, 10)

# contrast_next(-3)
# del(DATA[section][-1])
# del(DATA[section][-2]['contrast'])
# del(DATA[section][-2]['contrast_id'])
# DATA[section].append(
#     {
#       "section": 252,
#       "id": "252_18",
#       "transcription": "lɑsᵊrəχ",
#       "syllabified": "lɑs-ᵊrəχ",
#       "syllabified_incorrect": "lɑ-sə-rəχ"
#     }
# )


In [1502]:
mods = [
    {
      "section": 17,
      "id": "17_23",
      "irish": "lomaim",
      "transcription": "lɔmʷɩmʹ",
      "french": "je me dépouille, je deviens chauve",
      "future": "lomfad"
    },
    {
      "section": 177,
      "id": "177_25",
      "transcription": "sɔnə",
      "irish": "sona",
      "french": "heureux"
    },
    {
      "section": 177,
      "id": "177_23",
      "transcription": "krɔmʷɩmʹ",
      "irish": "cromaim",
      "french": "je courbe"
    },
   {
      "section": 177,
      "id": "177_24",
      "transcription": "fɔnəvər",
      "irish": "fonnmhar",
      "french": "désireux"
    },
]
modwith = ["lo̤mʷɩmʹ", "so̤nə", "kro̤mʷɩmʹ", "fo̤nəvər"]

section = 180
DATA[section] = []
for i in range(len(mods)):
    mod = mods[i]
    mod["section"] = section
    mod["compare_id"] = mod["id"]
    mod["id"] = f"{section}_{i + 1}"
    mod["transcription"] = modwith[i]
    DATA[section].append(mod)
